# Cleaning Antagonist Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#hiding warning messages
import warnings
warnings.filterwarnings("ignore")

#Reading in Summary ANTAGONIST CSV
antag = pd.read_csv('AID_720725_datatable_all.csv')
antag = antag.iloc[3:]
antag = antag.reset_index()
#antag.head(5)
#antag.dtypes

In [2]:
#Removing inconclusives
antag = antag.drop(antag.loc[antag['Activity Summary'] == 'inconclusive'].index)
antag = antag.drop(antag.loc[antag['Activity Summary'] == 'inconclusive agonist'].index)
antag = antag.drop(antag.loc[antag['Activity Summary'] == 'inconclusive antagonist'].index)
antag = antag.drop(antag.loc[antag['Activity Summary'] == 'inconclusive agonist (cytotoxic)'].index)
antag = antag.drop(antag.loc[antag['Activity Summary'] == 'inconclusive antagonist (cytotoxic)'].index)

#Keeping only CID, Activity Summary, Ratio Potency (uM), and Ratio Efficacy
antag = antag[['PUBCHEM_CID', 'Activity Summary', 'Ratio Potency (uM)', 'Ratio Efficacy (%)']]
antag.fillna(0, inplace = True)

In [3]:
##Dropping values with no CID
antag = antag.drop(antag.loc[antag['PUBCHEM_CID'] == 0].index)

#Dropping duplicates from original antag dataframe
antag_no_dups = antag.drop_duplicates(subset = 'PUBCHEM_CID', keep = False)

In [4]:
##Finding DF of all duplicates
antag_dups = antag[antag.duplicated('PUBCHEM_CID', keep = False)] 
antag_dups = antag_dups.sort_values(by = ['PUBCHEM_CID', 'Activity Summary'], ascending = True, inplace = False)

In [5]:
#Creating two seperate datasets for the three different cases (third case is all deleted)

In [8]:
##df_inactive = both activity scores inactive
###Drop all active agonists and active antagonists, and drop any values that are no longer dups.  
###This will leave CID dups with outcomes that are both inactive.
df_inactive = antag_dups.drop(antag_dups.loc[antag_dups['Activity Summary'] == 'active agonist'].index)
df_inactive = df_inactive.drop(df_inactive.loc[antag_dups['Activity Summary'] == 'active antagonist'].index)
df_inactive = df_inactive[df_inactive.duplicated(subset = 'PUBCHEM_CID', keep = False)]

#Only leaving one of each inactive value
df_inactive_clean = df_inactive.drop_duplicates(subset = 'PUBCHEM_CID', keep = 'first')
antag_dups

,PUBCHEM_CID,Activity Summary,Ratio Potency (uM),Ratio Efficacy (%)
4955,13.0,inactive,0,0
7326,13.0,inactive,0,0
15,51.0,inactive,0,0
4562,51.0,inactive,0,0
5313,79.0,inactive,0,0
7118,79.0,inactive,0,0
5986,135.0,inactive,0,0
10064,135.0,inactive,0,0
5682,174.0,inactive,0,0
7400,174.0,inactive,0,0


In [7]:
##df_active = both activity scores are active
###Drop all inactive values and drop any values that are no longer dups

df_active = antag_dups.drop(antag_dups.loc[antag_dups['Activity Summary'] == 'inactive'].index)
df_active = df_active[df_active.duplicated(subset = 'PUBCHEM_CID', keep = False)]

###Rank by ascending CID AND by ascending potency
###Drop duplicates and leave the FIRST value, which is the lowest potency!!!!!
df_active = df_active.sort_values(by = ['PUBCHEM_CID','Ratio Potency (uM)'], ascending = True, inplace = False)
df_active_clean = df_active.drop_duplicates(subset = 'PUBCHEM_CID', keep = 'first')

In [8]:
##df_disagree = the activity scores disagree
##We removed all the rest because they didn't disagree

In [10]:
#Combining antag_no_dups with df_active_clean and df_inactive_clean to create the final database with no dups
frames1 = [antag_no_dups, df_active_clean, df_inactive_clean]
#FINAL CLEAN DF
antag_clean = pd.concat(frames1)
antag_clean.shape

(6734, 4)

# Cleaning Agonist Data

In [13]:
#Reading in Summary AGONIST CSV
ag = pd.read_csv('AID_720719_datatable_all.csv')
ag = ag.iloc[4:]
ag = ag.reset_index()
#ag.head()

In [14]:
#Removing inconclusives
ag = ag.drop(ag.loc[ag['Activity Summary'] == 'inconclusive'].index)
ag = ag.drop(ag.loc[ag['Activity Summary'] == 'inconclusive agonist'].index)
ag = ag.drop(ag.loc[ag['Activity Summary'] == 'inconclusive agonist (fluorescent)'].index)
ag = ag.drop(ag.loc[ag['Activity Summary'] == 'inconclusive antagonist'].index)
ag = ag.drop(ag.loc[ag['Activity Summary'] == 'inconclusive agonist (cytotoxic)'].index)
ag = ag.drop(ag.loc[ag['Activity Summary'] == 'inconclusive antagonist (cytotoxic)'].index)

#Keeping only CID, Activity Summary, Ratio Potency (uM), and Ratio Efficacy
ag = ag[['PUBCHEM_CID', 'Activity Summary', 'Ratio Potency (uM)', 'Ratio Efficacy (%)']]
ag.fillna(0, inplace = True)

In [15]:
##Dropping values with no CID
ag = ag.drop(ag.loc[ag['PUBCHEM_CID'] == 0].index)

#Dropping duplicates from original ag dataframe
ag_no_dups = ag.drop_duplicates(subset = 'PUBCHEM_CID', keep = False)

In [16]:
##Finding DF of all duplicates
ag_dups = ag[ag.duplicated('PUBCHEM_CID', keep = False)] 
ag_dups = ag_dups.sort_values(by = ['PUBCHEM_CID', 'Activity Summary'], ascending = True, inplace = False)

In [17]:
#Creating two seperate datasets for the three different cases (third case is all deleted)

In [18]:
##df_inactive_ag = both activity scores inactive
###Drop all active agonists and active antagonists, and drop any values that are no longer dups.  
###This will leave CID dups with outcomes that are both active.
df_inactive_ag = ag_dups.drop(ag_dups.loc[ag_dups['Activity Summary'] == 'active agonist'].index)
df_inactive_ag = df_inactive_ag.drop(df_inactive_ag.loc[ag_dups['Activity Summary'] == 'active antagonist'].index)
df_inactive_ag = df_inactive_ag[df_inactive_ag.duplicated(subset = 'PUBCHEM_CID', keep = False)]

#Only leaving one of each inactive value
df_inactive_ag_clean = df_inactive_ag.drop_duplicates(subset = 'PUBCHEM_CID', keep = 'first')

In [19]:
##df_active_ag = both activity scores are active
###Drop all inactive values and drop any values that are no longer dups

df_active_ag = ag_dups.drop(ag_dups.loc[ag_dups['Activity Summary'] == 'inactive'].index)
df_active_ag = df_active_ag[df_active_ag.duplicated(subset = 'PUBCHEM_CID', keep = False)]

###Rank by descending CID AND by descending potency
###Drop duplicates and leave the FIRST value, which is the lowest potency!!!!!
df_active_ag = df_active_ag.sort_values(by = ['PUBCHEM_CID', 'Ratio Potency (uM)'], ascending = True, inplace = False)
df_active_ag_clean = df_active_ag.drop_duplicates(subset = 'PUBCHEM_CID', keep = 'first')

In [20]:
##df_disagree = the activity scores disagree
##We removed all the rest because they didn't disagree

In [21]:
#Combining antag_no_dups with df_active_clean and df_inactive_clean to create the final database with no dups
frames2 = [ag_no_dups, df_active_ag_clean, df_inactive_ag_clean]
#FINAL CLEAN DF
ag_clean = pd.concat(frames2)
ag_clean.shape
#(ag_clean['Activity Summary'] == 'inactive').sum()

(7405, 4)

# Combining Dataframes and Cleaning

In [26]:
final_concat.dtypes


PUBCHEM_CID           float64
Activity Summary       object
Ratio Potency (uM)     object
Ratio Efficacy (%)     object
dtype: object

In [22]:
#Combining antag_clean and ag_clean
frames3 = [antag_clean, ag_clean]
final_concat = pd.concat(frames3)

In [23]:
#Dropping duplicates from original final dataframe
final_no_dups = final_concat.drop_duplicates(subset = 'PUBCHEM_CID', keep = False)

In [24]:
##Finding DF of all duplicates
final_dups = final_concat[final_concat.duplicated('PUBCHEM_CID', keep = False)] 
final_dups = final_dups.sort_values(by = ['PUBCHEM_CID'], ascending = True, inplace = False)
final_dups = final_dups.reset_index(drop = True)

In [25]:
##df_inactive_final = both activity scores inactive
###Drop all active agonists and active antagonists, and drop any values that are no longer dups.  
###This will leave CID dups with outcomes that are both inactive.
df_inactive_final = final_dups.drop(final_dups.loc[final_dups['Activity Summary'] == 'active agonist'].index)
df_inactive_final = df_inactive_final.drop(df_inactive_final.loc[final_dups['Activity Summary'] == 'active antagonist'].index)
df_inactive_final = df_inactive_final[df_inactive_final.duplicated(subset = 'PUBCHEM_CID', keep = False)]

#Only leaving one of each inactive value
df_inactive_final_clean = df_inactive_final.drop_duplicates(subset = 'PUBCHEM_CID', keep = 'first')

In [23]:
##df_active_ag_final = both activity scores are active agonist
###Drop all inactive values and drop any values that are no longer dups
###Drop all active antagonist values

df_active_ag_final = final_dups.drop(final_dups.loc[final_dups['Activity Summary'] == 'inactive'].index)
df_active_ag_final = df_active_ag_final.drop(final_dups.loc[final_dups['Activity Summary'] == 'active antagonist'].index)
df_active_ag_final = df_active_ag_final[df_active_ag_final.duplicated(subset = 'PUBCHEM_CID', keep = False)]

###Rank by descending CID AND by descending potency
###Drop duplicates and leave the FIRST value, which is the lowest potency!!!!!
df_active_ag_final = df_active_ag_final.sort_values(by = ['PUBCHEM_CID', 'Ratio Potency (uM)'], ascending = True, inplace = False)
df_active_ag_final_clean = df_active_ag_final.drop_duplicates(subset = 'PUBCHEM_CID', keep = 'first')

In [24]:
##df_active_antag_final = both activity scores are active antagonists
###Drop all inactive values and drop any values that are no longer dups
###Drop all active agonist values

df_active_antag_final = final_dups.drop(final_dups.loc[final_dups['Activity Summary'] == 'inactive'].index)
df_active_antag_final = df_active_antag_final.drop(final_dups.loc[final_dups['Activity Summary'] == 'active agonist'].index)
df_active_antag_final = df_active_antag_final[df_active_antag_final.duplicated(subset = 'PUBCHEM_CID', keep = False)]

###Rank by descending CID AND by descending potency
###Drop duplicates and leave the FIRST value, which is the lowest potency!!!!!
df_active_antag_final = df_active_antag_final.sort_values(by = ['PUBCHEM_CID', 'Ratio Potency (uM)'], ascending = True, inplace = False)
df_active_antag_final_clean = df_active_antag_final.drop_duplicates(subset = 'PUBCHEM_CID', keep = 'first')

In [25]:
#getting lists of CID's of rows that fall into othr categories
inactive_list = df_inactive_final['PUBCHEM_CID'].tolist()
active_ag_list = df_active_ag_final['PUBCHEM_CID'].tolist()
active_antag_list = df_active_antag_final['PUBCHEM_CID'].tolist()
#combining lists
index_list = inactive_list + active_ag_list + active_antag_list

#Dropping those CID's from full dups dataframe final_dups
##This will create a dataframe with CID pairs that are either inactive/active or active/active
##Any remaining active/active CID pairs must be active agonist/antagonist pairs, and are inconclusive
##We will delete those values
final_dups_newindex = final_dups.set_index('PUBCHEM_CID')
df_disagree = final_dups_newindex.drop(labels = index_list)
df_disagree = df_disagree.reset_index()

In [26]:
#Sorting by CID
df_disagree = df_disagree.sort_values(by = ['PUBCHEM_CID', 'Activity Summary'], ascending = True, inplace = False)

In [27]:
#We remove all inactive values (BC we are keeping active)
df_disagree = df_disagree.drop(df_disagree.loc[df_disagree['Activity Summary'] == 'inactive'].index)
#We then delete any remaining duplicates, because that means they were active agonist/antagonist pairs
df_disagree_clean = df_disagree.drop_duplicates(subset = 'PUBCHEM_CID', keep = False)

In [28]:
#We combine all the pieces into the full cleaned combined dataframe
frames4 = [final_no_dups, df_inactive_final_clean, df_active_ag_final_clean, df_active_antag_final_clean, df_disagree_clean]
final_clean = pd.concat(frames4)

In [29]:
#Writing to csv
final_clean.to_csv('Final_Merged_CSV', index = False)


In [32]:
final = pd.read_csv('Final_Merged_CSV')